In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import random as rnd
import copy

from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from shutil import copy2
from utils import augment_image

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
origin_path = '../datasets/UTKFace_origin/'
debiased_path = '../datasets/UTKFace_debiased/'
images = os.listdir(origin_path)
count = 0
conflicted = []
for i in images:
    splited_i = i.split('_')
    if len(splited_i) < 4:
        count += 1
        print(i)
        conflicted.append(i)
        
for i in conflicted:
    images.remove(i)

39_1_20170116174525125.jpg.chip.jpg
61_1_20170109142408075.jpg.chip.jpg
61_1_20170109150557335.jpg.chip.jpg


In [3]:
ages = []
genders = []
races = []
for i in images:
    split = i.split('_')
    ages.append(int(split[0]))
    genders.append(int(split[1]))
    races.append(int(split[2]))

In [4]:
pd_images = pd.Series(list(images), name = 'Images')
pd_ages = pd.Series(list(ages), name = 'Ages')
pd_genders = pd.Series(list(genders), name = 'Genders')
pd_races = pd.Series(list(races), name = 'Races')

df = pd.concat([pd_images, pd_ages, pd_genders, pd_races], axis=1)
df

,Images,Ages,Genders,Races
0,100_0_0_20170112213500903.jpg.chip.jpg,100,0,0
1,100_0_0_20170112215240346.jpg.chip.jpg,100,0,0
2,100_1_0_20170110183726390.jpg.chip.jpg,100,1,0
3,100_1_0_20170112213001988.jpg.chip.jpg,100,1,0
4,100_1_0_20170112213303693.jpg.chip.jpg,100,1,0
...,...,...,...,...
23700,9_1_3_20161220222856346.jpg.chip.jpg,9,1,3
23701,9_1_3_20170104222949455.jpg.chip.jpg,9,1,3
23702,9_1_4_20170103200637399.jpg.chip.jpg,9,1,4
23703,9_1_4_20170103200814791.jpg.chip.jpg,9,1,4


In [6]:
races_genders = np.array([[0]*len(df["Genders"].value_counts())] * len(df["Races"].value_counts()))
racesGenders_age = {}
images_browser = {}

for i in range(len(images)):
    age = ages[i]
    gender = genders[i]
    race = races[i]
    image = images[i]
    
    races_genders[race][gender] += 1
    if (race,gender) not in racesGenders_age:
        racesGenders_age[(race,gender)] = [0]*pd_ages.max()
    racesGenders_age[(race,gender)][age-1] += 1
    if (race,gender,age) not in images_browser:
        images_browser[(race,gender,age)] = []
    images_browser[(race,gender,age)].append(image)
    
racesGenders_age

{(0, 0): [143,
  55,
  41,
  50,
  28,
  14,
  39,
  45,
  114,
  67,
  21,
  25,
  30,
  41,
  58,
  109,
  40,
  32,
  17,
  46,
  32,
  44,
  39,
  69,
  110,
  268,
  93,
  136,
  97,
  145,
  55,
  114,
  26,
  89,
  233,
  107,
  70,
  61,
  69,
  186,
  38,
  86,
  54,
  19,
  146,
  39,
  67,
  56,
  44,
  131,
  58,
  101,
  97,
  145,
  113,
  76,
  43,
  136,
  43,
  89,
  55,
  45,
  53,
  23,
  99,
  19,
  42,
  46,
  12,
  66,
  13,
  27,
  22,
  16,
  65,
  15,
  8,
  30,
  8,
  58,
  6,
  17,
  6,
  1,
  30,
  7,
  3,
  6,
  2,
  31,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0],
 (0, 1): [144,
  39,
  74,
  57,
  47,
  40,
  32,
  127,
  17,
  39,
  28,
  53,
  30,
  56,
  43,
  73,
  37,
  116,
  28,
  68,
  46,
  46,
  77,
  133,
  132,
  261,
  78,
  118,
  93,
  134,
  46,
  123,
  38,
  58,
  143,
  72,
  53,
  54,
  59,
  87,
  25,
  44,
  24,
  13,
  61,
  24,
  26,
  29,
  47,
 

: 

In [ ]:
def get_minority_age(minority_race, minority_gender):
    aux = np.where(racesGenders_age == 0, 10000000, races)

def get_majority_age(majority_race, majority_gender):
    valid_majority_ages = [age for age in range(120) if (majority_race,majority_gender,age+1) in images_browser]
    
    aux = []
    for i in range(len(genders_age[majority_gender])):
        if i in valid_majority_ages:
            aux.append(genders_age[majority_gender][i])
        else:
            aux.append(0)
    
    majority_age = np.argmax(aux) + 1 
    
    return majority_age